In [146]:
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient
import sys
from pathlib import Path
from tqdm import tqdm
import json
import random
import bson

sys.path.append(str(Path("..").resolve()))
from src import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Final touches: recover the lost Consistency due to seed
ℹ️ This notebook require the execution of [`1.2-Seeding_Database`](1.2-Seeding_Database.ipynb). Or you can just load the snapshop as in the following cell.

## Load post-preprocessing data
The following cell reset the `omero_museum` database loading the `3_seeded` snapshot.

In [147]:
%%capture
MongoClient("mongodb://localhost:27017/").drop_database("omero_museum")
!mongorestore --host localhost:27017 --drop --db omero_museum  "../backup/3_seeded/omero_museum"

In [148]:
connector = MongoDBConnector("omero_museum")
db = connector.db

The collections of the omero_museum db are:
----------------------------------------
[activities]: _id capacity duration enrolled room startDate ticketIds workshopTitle
[artworks]: _id authorIds comments_star_1 comments_star_2 comments_star_3 comments_star_4 comments_star_5 date description donationState donatorId isOriginal locationName materials period sellerId size tecniques trade type
[authors]: _id birth_date gender home_town name surname
[departments]: _id floor free_spots room
[limited_events]: _id artist artworkTitles authorIds capacity description endDate roomName startDate theme ticketIds type
[messages]: _id content customerId deliveryStatus length sentDate type uri
[roles]: _id activityIds birth_date chatIds curriculum date_start department desk email english_cert gender hometown name password phone_number salary shifts surname type username
[rooms]: _id floor
[suppliers]: _id email iban isMuseum isState name phoneNum surveyIds
[visitors]: _id birthDate chat comments donati

## Artworks' comments' rating

In [149]:
for star in [1, 2, 3, 4, 5]:
    db.artworks.update_many({}, {"$set": {f"comments_star_{star}.$[].rating": star}})

## Visitors' `is_customer`

In [150]:
customer_fields = [
    "surname",
    "name",
    "donationsIds",
    "salesIds",
    "phoneNum",
    "email",
    "tradeIds",
    "chat",
    "comments",
]
db.visitors.update_many(
    {"is_customer": False},
    [
        {"$unset": customer_fields},
        {
            "$set": {
                "tickets": {"$slice": ["$tickets", 1]},
                "surveys": {"$slice": ["$surveys", 1]},
            }
        },
    ],
)

UpdateResult({'n': 48, 'nModified': 48, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

## Ticket's missing `id`

In [151]:
""" for doc in db.visitors.find({}):
    if "tickets" in doc:
        db.visitors.update_one(
            {"_id": doc["_id"]},
            {
                "$set": {
                    "tickets": [{**t, "_id": bson.ObjectId()} for t in doc["tickets"]]
                }
            },
        ) """

' for doc in db.visitors.find({}):\n    if "tickets" in doc:\n        db.visitors.update_one(\n            {"_id": doc["_id"]},\n            {\n                "$set": {\n                    "tickets": [{**t, "_id": bson.ObjectId()} for t in doc["tickets"]]\n                }\n            },\n        ) '

## Fix Event-Ticket broken reference

In [152]:
""" ids = {
    doc["_id"]: [t["_id"] for t in doc["tickets"]]
    for doc in db.visitors.find({}, {"_id": 1, "tickets": 1})
    if doc.get("tickets")
}

for c in [db.activities, db.limited_events]:
    for doc in c.find({}):
        if "ticketIds" in doc:
            c.update_one(
                {"_id": doc["_id"]},
                {
                    "$set": {
                        "ticketIds": [
                            random.choice(random.choice(list(ids.values())))
                            for _ in doc["ticketIds"]
                        ]
                    }
                },
            ) """

' ids = {\n    doc["_id"]: [t["_id"] for t in doc["tickets"]]\n    for doc in db.visitors.find({}, {"_id": 1, "tickets": 1})\n    if doc.get("tickets")\n}\n\nfor c in [db.activities, db.limited_events]:\n    for doc in c.find({}):\n        if "ticketIds" in doc:\n            c.update_one(\n                {"_id": doc["_id"]},\n                {\n                    "$set": {\n                        "ticketIds": [\n                            random.choice(random.choice(list(ids.values())))\n                            for _ in doc["ticketIds"]\n                        ]\n                    }\n                },\n            ) '

## Roles and chats

In [153]:
""" visistors_with_chat = db.visitors.find({"chat": {"$exists": 1}}) """

' visistors_with_chat = db.visitors.find({"chat": {"$exists": 1}}) '

## Dump Final Database

In [154]:
%%capture
!mongodump --host localhost:27017 --db omero_museum --out "../backup/4_consistent"